In [9]:
%pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 28.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
# Generate Terms Related to Query
question = "Identify all British Columbia regulations that contain clauses that specify how the regulation applies to goods or services originating outside the province"

In [3]:
# Get nodes with text that match the above terms.
from JEDI_Search_Task.Neo4jRetrieval import Neo4jRetrieval

NEO4J_URI = "bolt://" + "neo4j:7687" 
NEO4J_USERNAME = "admin"  
NEO4J_PASSWORD = "admin" 

# TODO: Remove this and replace with terms from previous step
terms = ["trade", "provincial law", "commerce", "regulation", "market"]

neo4j_worker = Neo4jRetrieval(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)
nodes = neo4j_worker.search_many(terms)
print(len(nodes), "nodes found")
neo4j_worker.close()

14055 nodes found


In [23]:
import json
import os
from JEDI_Search_Task.BedrockQuery import BedrockQuery

# Replace this with updated keys if necessary
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY =  os.getenv("AWS_SECRET_ACCESS_KEY")
bedrock = BedrockQuery(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)

def create_prompt(question, node):
  return f"""
    Your task is to determine if the following object is related to the question: "{question}".

    Object: {node}

    If the object is related, "1". If it is not related, "0".
    Do not return any explanation, just the number.

    For example:
    Question: "What is the capital of France?"
    Object: {{ "text": "Paris is the capital of France." }}
    Response: 1
    """

# Filter out nodes that are not related to the query using LLM.
# For each node, decide if it is related to the query.
related_nodes = []
for node in nodes:
    # Use an LLM to determine if the node is relevant.
    prompt = create_prompt(question, node)
    bedrock_response = bedrock.get_response(prompt)
    print("response", bedrock_response.strip())
    try:
        if int(bedrock_response.strip()):
            related_nodes.append(node)
            print("good",node.get("text"))
        else:
            print("bad", node.get("text"))
    except:
        continue
print(len(related_nodes), "related nodes found")
# Save the related nodes to a file.
with open("related_nodes.jsonl", "w") as f:
    for node in related_nodes:
        f.write(json.dumps(node) + "\n")

response 0
bad under the climate change accountability act ; h to encourage the switching from one kind of energy source or use to another that decreases greenhouse gas emissions in british columbia ; i to encourage communities to reduce greenhouse gas emissions and use energy efficiently ; j to reduce waste by encouraging the use of waste heat, biogas and biomass ; k to encourage economic development and the creation and retention of jobs ; l to foster the development of first nation and rural communities through the use and development of clean or renewable resources ; m to maximize the value, including the incremental value of the resources being clean or renewable resources, of british columbia's generation and transmission assets for the benefit of british columbia ; n to be a net exporter of electricity from clean or renewable resources with the intention of benefiting all british columbians and reducing greenhouse gas emissions in regions in which british columbia trades electri

KeyboardInterrupt: 

In [ ]:
# Generate report with the remaining information.